In [31]:
import os

TRANSACTION_FILE = "transactions.txt"
USER_FILE = "users.txt"

In [32]:
# Record User to file.
def record_user(account):
    with open(USER_FILE, "a") as file:
        file.write(f"{account['account_title']},{account['cnic']},{account['phone']},{account['dob']},{account['balance']}\n")

In [54]:
# Update balance in USERS.txt
def update_user_balance(cnic_filter, new_balance):
    if not os.path.exists(USER_FILE):
        print("User list file not found.")
        return

    with open(USER_FILE, "r") as file:
        users = file.readlines()

    updated_users = []
    for user in users:
        title, cnic, phone, dob, balance = user.strip().split(",")
        if cnic_filter == cnic:
            updated_users.append(f"{title},{cnic},{phone},{dob},{new_balance:.2f}\n")
        else:
            updated_users.append(user)

    with open(USER_FILE, "w") as file:
        file.writelines(updated_users)

In [48]:
# Load User with CNIC
def load_user(cnic_filter):
    if not os.path.exists(USER_FILE):
        print("No user data found.")
        return None

    with open(USER_FILE, "r") as file:
        users = file.readlines()
        for user in users:
            title, cnic, phone, dob, balance = user.strip().split(",")
            if cnic_filter == cnic:
                # print(f"Account Title: {title} (CNIC: {cnic}) loaded with balance ${balance}.")
                return {
                    "account_title": title,
                    "cnic": cnic,
                    "phone": phone,
                    "dob": dob,
                    "balance": float(balance),
                    "transactions": []
                }
    print(f"No user found with cnic {cnic_filter}.")
    return None

In [70]:
# Load User with CNIC
def load_account_holder():
    if not os.path.exists(USER_FILE):
        print("No user data found.")
        return None

    with open(USER_FILE, "r") as file:
        users = file.readlines()
        if not users:
            print('No record for account holder found')
        else:
            for user in users:
                title, cnic, phone, dob, balance = user.strip().split(",")
                print(f'{title} - {cnic} - {phone} - {dob} - {balance}')

In [43]:
# Record Transaction to file.
# transaction_type: Deposit, Withdrawal
def record_transaction(title, cnic, transaction_type, amount, balance):
    with open(TRANSACTION_FILE, "a") as file:
        file.write(f"{title},{cnic},{transaction_type},{amount},{balance}\n")

In [58]:
# Create an account
def create_account(title, cnic, dob, phone = "",initial_balance=0.0):
    if (title == '' or cnic == ''):
        print('ERROR: Account title and CNIC is required field to create account')
        return
    account = {
        "account_title": title,
        "cnic": cnic,
        "phone": phone,
        "dob": dob,
        "balance": initial_balance,
        "transactions": []
    }
    record_user(account)
    print(f"====== Congratulations! Account for {title} created with balance ${initial_balance}. ======")
    return account

In [62]:
# Deposit function
def deposit(account, amount):
    if amount <= 0:
        print("Error: Deposit amount must be positive.")
        return account
    
    account["balance"] += amount
    account["transactions"].append(("Deposit", amount, account["balance"]))
    record_transaction(account["account_title"],account["cnic"], "Deposit", amount, account["balance"])
    update_user_balance(account["cnic"], account["balance"])
    print(f"====== {account['account_title']} deposited ${amount}. New balance: ${account['balance']:.2f}. ======")
    return account

In [63]:
# Withdraw function
def withdraw(account, amount):
    if amount <= 0:
        print("Error: Withdrawal amount must be positive.")
        return account
    
    if amount > account["balance"]:
        print("Error: Insufficient balance.")
        return account
    
    account["balance"] -= amount
    account["transactions"].append(("Withdrawal", amount, account["balance"]))
    record_transaction(account["account_title"], account["cnic"], "Withdrawal", amount, account["balance"])
    update_user_balance(account["cnic"], account["balance"])
    print(f"====== {account['account_title']} withdrew ${amount}. New balance: ${account['balance']:.2f}. ======")
    return account

In [68]:
# Load Statement Function
def load_statement(account):
    if not os.path.exists(TRANSACTION_FILE):
        print("No transactions data found.")
        return None
        
    with open(TRANSACTION_FILE, "r") as file:
        transactions = file.readlines()
        user_transactions = []
        for transaction in transactions:
            title, cnic, transaction_type, amount, balance = transaction.strip().split(",")
            if account['cnic'] == cnic:
                user_transactions.append({
                    "title": title,
                    "transaction_type": transaction_type,
                    "amount": float(amount),
                    "balance": float(balance)
                })
        if not user_transactions:
            print(f"No transactions found for account holder {account['account_title']}.")
            return None
        else:
            print(f"\nTransactions for {account['account_title']}:")
            for transaction in user_transactions:
                print(f"{transaction['transaction_type']} - Amount: {transaction['amount']} - Balance: {transaction['balance']}")

In [71]:
while True:
    print('''
1 > Create Account
2 > Deposit Amount
3 > Withdraw Amount
4 > Check Balance
5 > Statement
6 > List Account Holders
7 > Exit
''')
    try:
        operation = input('Select what operation to perform: ')
        
        if operation == '1':
            print('**** Creating Account ****')
            account_title = input('Enter your name: ')
            cnic = input('Enter your CNIC (without dash): ')
            dob = input('Enter your Date of Birth (dd/mm/yyyy): ')
            phone = input('Enter your Phone Number: ')
            create_account(account_title, cnic, dob, phone)
        
        elif operation == '2':
            print('**** Deposit Amount ****')
            cnic = input('Enter your CNIC: ')
            account_data = load_user(cnic)
            if account_data is not None:
                amount = float(input('Enter amount to deposit: '))
                deposit(account_data, amount)
        
        elif operation == '3':
            print('**** Withdraw Amount ****')
            cnic = input('Enter your CNIC: ')
            account_data = load_user(cnic)
            if account_data is not None:
                amount = float(input('Enter amount to withdraw: '))
                withdraw(account_data, amount)
        
        elif operation == '4':
            print('**** Checking Balance ****')
            cnic = input('Enter your CNIC: ')
            account_data = load_user(cnic)
            if account_data is not None:
                print(f'Account Title: {account_data["account_title"]} has ${account_data["balance"]} balance in account.) ')
        
        elif operation == '5':
            print('**** Statement ****')
            cnic = input('Enter your CNIC: ')
            account_data = load_user(cnic)
            if account_data is not None:
                load_statement(account_data)
        
        elif operation == '6':
            print('**** List of Account Holders ****')
            load_account_holder()
        
        elif operation == '7':
            print('Exit')
            break
        
        else:
            print('Invalid option. Please select a valid operation.')
    
    except ValueError:
        print('Invalid input. Please enter a number.')


1 > Create Account
2 > Deposit Amount
3 > Withdraw Amount
4 > Check Balance
5 > Statement
6 > List Account Holders
7 > Exit



Select what operation to perform:  6


**** List of Account Holders ****
Muhammad Zeeshan - 4230180519111 - 03122309493 - 03/01/1995 - 90.00
Muhammad Ayaz - 4230111215887 - 0123232323 - 01/01/1900 - 150.00

1 > Create Account
2 > Deposit Amount
3 > Withdraw Amount
4 > Check Balance
5 > Statement
6 > List Account Holders
7 > Exit



Select what operation to perform:  7


Exit
